## Modèle BERT

In [1]:
from helpers import *

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]  
df = pd.read_csv("./../input/training.1600000.processed.noemoticon.csv", header=None, names=cols)

# df = df.sample(n=1000, random_state=42)
len_df = len(df)
print(df.shape)
df.head()

(1600000, 6)


,sentiment,timestamp,date,query,username,comment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 1600000/1600000 [00:02<00:00, 571854.79it/s]


(1469990, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,awww thats a bummer you shoulda got david carr...,19,16
1,is upset that he can't update his Facebook by ...,0,is upset that he cant update his facebook by t...,21,21
2,@Kenichan I dived many times for the ball. Man...,0,i dived many times for the ball managed to sav...,18,16
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire,10,10
4,"@nationwideclass no, it's not behaving at all....",0,no its not behaving at all im mad why am i her...,21,20


- les données sont préparées (input ids, attention mask) pour mettre en oeuvre une démarche d’embedding pour le “modèle BERT”
- une démarche optionnelle d’embedding via USE est mise en oeuvre.


In [5]:
from transformers import BertTokenizer, TFBertForSequenceClassification  
from transformers import InputExample, InputFeatures
from transformers import BertTokenizerFast
import tensorflow as tf  
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

/Users/m/dev/6be709c0/oc-ai-engineer/OC/P7/output/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')  


In [7]:
# Assume preprocess_text is defined elsewhere  
df_bert_approach = df.copy()

In [8]:
df_bert_approach = df_bert_approach.reset_index(drop=True)
X_train, X_test, y_train, y_test = train_test_split(df_bert_approach["comment_clean"], df_bert_approach['sentiment'], test_size=0.3, random_state=42)  
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)  

In [9]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)  
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)  

In [10]:
class TweetDataset(Dataset):  
    def __init__(self, encodings, labels=None):  
        self.encodings = encodings  
        self.labels = labels  
  
    def __getitem__(self, idx):  
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}  
        if self.labels:  
            item['labels'] = torch.tensor(self.labels[idx])  
        return item  
  
    def __len__(self):  
        return len(self.encodings.input_ids)  

In [11]:
# Convert to torch Dataset  
train_dataset = TweetDataset(train_encodings, y_train.tolist())  
val_dataset = TweetDataset(val_encodings, y_val.tolist())

In [12]:
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)  
test_dataset = TweetDataset(test_encodings, y_test.tolist())

In [13]:
# Load DistilBERT model  
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(df_bert_approach['sentiment'].tolist())))  

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# !pip install accelerate -U

In [15]:
# Define training arguments  
training_args = TrainingArguments(  
  output_dir='./results',          # output directory  
  num_train_epochs=1,              # total number of training epochs  
  per_device_train_batch_size=256,   # batch size per device during training, adjust based on your GPU  
  per_device_eval_batch_size=256,   # batch size for evaluation  
  warmup_steps=500,                # number of warmup steps for learning rate scheduler  
  weight_decay=0.01,               # strength of weight decay  
  logging_dir='./logs',            # directory for storing logs  
  logging_steps=10,  
  evaluation_strategy="epoch",  
  save_strategy="epoch",           # Ensure the save strategy matches the evaluation strategy  
  load_best_model_at_end=True,  
)  

In [16]:
# Initialize the Trainer  
trainer = Trainer(  
  model=model,  
  args=training_args,  
  train_dataset=train_dataset,  
  eval_dataset=val_dataset  
)  

In [17]:
import torch

In [18]:
# Train the model  
trainer.train()  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

{'loss': 0.6985, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/4020 [00:56<2:54:16,  2.61s/it]

{'loss': 0.6981, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  1%|          | 30/4020 [01:22<2:54:22,  2.62s/it]

...

                                                     
100%|██████████| 4020/4020 [3:30:16<00:00,  2.63s/it]

{'eval_loss': 0.34091079235076904, 'eval_runtime': 891.3641, 'eval_samples_per_second': 247.371, 'eval_steps_per_second': 0.967, 'epoch': 1.0}


100%|██████████| 4020/4020 [3:30:18<00:00,  3.14s/it]

{'train_runtime': 12618.3311, 'train_samples_per_second': 81.547, 'train_steps_per_second': 0.319, 'train_loss': 0.37649726393210947, 'epoch': 1.0}


TrainOutput(global_step=4020, training_loss=0.37649726393210947, metrics={'train_runtime': 12618.3311, 'train_samples_per_second': 81.547, 'train_steps_per_second': 0.319, 'train_loss': 0.37649726393210947, 'epoch': 1.0})

In [19]:
predictions = trainer.predict(test_dataset) 

100%|██████████| 862/862 [13:59<00:00,  1.03it/s]


In [20]:
predictions

PredictionOutput(predictions=array([[-1.4068747 ,  1.6896713 ],
       [-0.81753546,  0.9883083 ],
       [ 0.5470752 , -0.5866696 ],
       ...,
       [-1.2218585 ,  1.4904003 ],
       [-1.3381128 ,  1.558924  ],
       [ 0.2820865 , -0.2754275 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 1, 1, 1]), metrics={'test_loss': 0.3410319685935974, 'test_runtime': 840.5974, 'test_samples_per_second': 262.312, 'test_steps_per_second': 1.025})

In [21]:
from scipy.special import softmax  

# Getting the raw predictions  
raw_predictions = predictions.predictions  
  
# Assuming a classification task, apply softmax if needed (for getting probabilities)  
prob_predictions = softmax(raw_predictions, axis=1)  
  
# For getting the predicted class indices  
predicted_classes = np.argmax(prob_predictions, axis=1)  

In [22]:
accuracy = accuracy_score(y_test, predicted_classes)
print(accuracy)

0.8502850353062825
